# Andre Marroquin
# Serio orellana 
# Rodirgo mansilla
# carlos valladares

## 1. Situación Problemática:

En regiones agrícolas vulnerables, los periodos de **sequía** y otros **estresores bióticos/abióticos** (plagas, enfermedades, deficiencia nutricional, viento, inundación) impactan el rendimiento de los cultivos. La identificación **temprana y en terreno** del tipo y magnitud del daño permite **priorizar asistencia** y **optimizar decisiones** (riego, fertilización, control de plagas). Sin embargo, el monitoreo manual es **lento**, **costoso** y **subjetivo**.


## Problema científico:

¿Cómo **caracterizar** y **limpiar** el dataset para obtener una visión clara y confiable de la **distribución** de daños por **temporada** y **etapa de crecimiento**, identificando **patrones**, **valores atípicos** y **faltantes**, que sirvan de base para modelos posteriores de **clasificación/regresión** del daño?


## Objetivo general
Desarrollar un **análisis exploratorio reproducible** y un **pipeline de preprocesamiento** para el dataset CGIAR *Drought on the Ground* que permita comprender la **estructura**, **calidad** y **patrones** de los datos (etapas fenológicas y tipos/extensión de daño), y deje listos **artefactos limpios y documentados** para fases posteriores de **modelado, evaluación y despliegue**.

### Objetivos específicos 

1) **Inventario y calidad de datos**  
   - Elaborar un **diccionario de datos** con *nombre*, *tipo*, *% de faltantes*, *# de valores únicos* y *ejemplo* por columna para `Train.csv` y `Test.csv`.  

2) **Estandarización de variables clave**  
    - Normalizar **GROWTH_STAGE** a códigos `F/M/S/V` y nombres (`Flowering/Maturity/Sowing/Vegetative`).  
    - Derivar/estandarizar **SEASON** (e.g., `LR2020`, `SR2021`) desde columna explícita o **ID**.  

3) **Normalización de etiquetas de daño**  
   - Convertir columnas de daño (`DR, DS, FD, G, ND, PS, WD, WN`) a **numérico** y asegurar valores en **[0, 100]** con pasos de **10**; reportar violaciones.  

4) **Análisis exploratorio descriptivo**  
   - Calcular estadísticas (media, mediana, desviación, IQR) para cada etiqueta de daño y generar **gráficos**:  
     - Histogramas (cada etiqueta),  
     - Boxplots (cada etiqueta),  
     - Barras de frecuencia para `SEASON`, `GROWTH_STAGE`, `DOMINANT_DAMAGE`,  
     - **Matriz de correlación** entre etiquetas.  

5) **Cruces y patrones**  
   - Construir y analizar **crosstabs** `GROWTH_STAGE × SEASON` y `DOMINANT_DAMAGE × SEASON`, describiendo **≥ 5 hallazgos** concretos (desbalances, patrones estacionales/fenológicos).  

## 2. Investigación preliminar

- **Contexto agrícola**: La sequía reduce disponibilidad hídrica y afecta procesos fisiológicos (germinación, crecimiento vegetativo, floración), generando pérdidas.  
- **Dimensión fenológica**: Las **etapas de crecimiento (S, V, F, M)** condicionan la **susceptibilidad** y el impacto de cada daño.  
- **Tipos de daño (DR, DS, FD, G, ND, PS, WD, WN)**:  
  - **DR** (Drought), **FD** (Flood): estresores abióticos con patrones visuales diferentes.  
  - **DS** (Disease), **PS** (Pest), **WD** (Weed): bióticos y de manejo.  
  - **ND** (Nutrient Deficient): clorosis/necrosis; **WN** (Wind): daño físico; **G** (Good): referencia de buen estado.  
- **Etiquetado**: Extensión del daño en **porcentajes** (0–100 en pasos de 10).  
- **Implicación para análisis**:  
  - La **distribución por temporada** (LR/SR, 2020/2021) puede influir en patrones de daño.  
  - La **etapa** modula la **apariencia y severidad** de daños, clave para interpretar resultados y posteriormente diseñar modelos.

## 3. Análisis inicial del problema y los datos disponibles
- **Archivos**: `Train.csv` (con etiquetas), `Test.csv` (sin etiquetas), `SampleSubmission.csv` (formato de envío).  
- **Variables esperadas**:
  - Identificador de imagen (p. ej., `ID` o similar).
  - **Temporada** (p. ej., `LR2020`, `SR2021`), explícita o inferible desde el ID.
  - **Etapa de crecimiento** (`F/M/S/V` o nombre completo).
  - **Columnas de daño**: {`DR`,`DS`,`FD`,`G`,`ND`,`PS`,`WD`,`WN`} con porcentajes 0..100 (pasos de 10).
- **Riesgos de calidad**:
  - Falta de columnas explícitas (temporada/etapa).
  - Tipos de datos incorrectos (strings en columnas numéricas).
  - Etiquetas con múltiplos de 10 y ceros frecuentes → distribuciones discretas.
  - Diferencias de distribución por **temporada** que pueden sesgar análisis.



# IMPORTS Y CONFIGURACIÓN

In [63]:
# librerías
import os
import sys
import math
import json
import textwrap
import warnings
from pathlib import Path
from typing import List, Dict, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

warnings.filterwarnings("ignore")

# config global
DATA_DIR = Path(".") 
TRAIN_CSV = DATA_DIR / "Train.csv"
TEST_CSV = DATA_DIR / "Test.csv"
SAMPLE_CSV = DATA_DIR / "SampleSubmission.csv"

OUTPUT_DIR = Path("./outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# config pandas
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

print("RUTAS:")
print(" - TRAIN:", TRAIN_CSV)
print(" - TEST:", TEST_CSV)
print(" - SAMPLE:", SAMPLE_CSV)
print("OUTPUT_DIR:", OUTPUT_DIR.resolve())


RUTAS:
 - TRAIN: Train.csv
 - TEST: Test.csv
 - SAMPLE: SampleSubmission.csv
OUTPUT_DIR: C:\Users\andre\OneDrive\Desktop\PY2-DS\outputs


# CARGA DE DATOS Y VISTAS RÁPIDAS

In [64]:
# carga de datos
def CARGAR_CSVS():
    assert TRAIN_CSV.exists(), f"NO SE ENCONTRÓ: {TRAIN_CSV}"
    assert TEST_CSV.exists(), f"NO SE ENCONTRÓ: {TEST_CSV}"
    assert SAMPLE_CSV.exists(), f"NO SE ENCONTRÓ: {SAMPLE_CSV}"
    train_df = pd.read_csv(TRAIN_CSV)
    test_df = pd.read_csv(TEST_CSV)
    sample_df = pd.read_csv(SAMPLE_CSV)
    return train_df, test_df, sample_df

TRAIN_DF, TEST_DF, SAMPLE_DF = CARGAR_CSVS()

print("SHAPES:")
print(" - TRAIN:", TRAIN_DF.shape)
print(" - TEST :", TEST_DF.shape)
print(" - SAMPLE:", SAMPLE_DF.shape)

print("\nCOLUMNAS TRAIN:")
print(list(TRAIN_DF.columns))

print("\nHEAD TRAIN:")
display(TRAIN_DF.head(3))

print("\nHEAD TEST:")
display(TEST_DF.head(3))

print("\nHEAD SAMPLE:")
display(SAMPLE_DF.head(3))


SHAPES:
 - TRAIN: (26068, 6)
 - TEST : (8663, 5)
 - SAMPLE: (8663, 2)

COLUMNAS TRAIN:
['ID', 'filename', 'growth_stage', 'damage', 'extent', 'season']

HEAD TRAIN:


,ID,filename,growth_stage,damage,extent,season
0,ID_1S8OOWQYCB,L427F01330C01S03961Rp02052.jpg,S,WD,0,SR2020
1,ID_0MD959MIZ0,L1083F00930C39S12674Ip.jpg,V,G,0,SR2021
2,ID_JRJCI4Q11V,24_initial_1_1463_1463.JPG,V,G,0,LR2020



HEAD TEST:


,ID,filename,growth_stage,damage,season
0,ID_ROOWKB90UZ,L122F00315C01S02151Rp04021.jpg,V,WD,SR2020
1,ID_PTEDRY0CYM,L1089F03254C01S08845Rp25119.jpg,F,WD,LR2021
2,ID_5WJXDV96R4,L365F01913C39S12578Rp42918.jpg,V,WD,SR2021



HEAD SAMPLE:


,ID,extent
0,ID_KJ12GE2U80,0
1,ID_W33POE3DBX,0
2,ID_1DZ7VKQTS9,0


# DETECCIÓN DE CAMPOS Y RESÚMENES

In [65]:
# detección de campos
DAMAGE_CODES = ["DR","DS","FD","G","ND","PS","WD","WN"]
GROWTH_SET_SHORT = {"F","M","S","V"}
GROWTH_MAP = {
    "F": "Flowering", "M": "Maturity", "S": "Sowing", "V": "Vegetative",
    "FLOWERING":"Flowering","MATURITY":"Maturity","SOWING":"Sowing","VEGETATIVE":"Vegetative"
}

# resumen de columnas
def COLS_OVERVIEW(DF: pd.DataFrame) -> pd.DataFrame:
    INFO = []
    for C in DF.columns:
        S = DF[C]
        INFO.append({
            "COLUMN": C,
            "DTYPE": str(S.dtype),
            "NON_NULL": int(S.notna().sum()),
            "NULLS": int(S.isna().sum()),
            "UNIQUE": int(S.nunique()),
            "EXAMPLE": (S.dropna().iloc[0] if S.notna().any() else None),
        })
    OUT = pd.DataFrame(INFO).sort_values(["DTYPE","COLUMN"]).reset_index(drop=True)
    return OUT

# extraer temporada desde ID
def EXTRAER_SEASON_DESDE_ID(V):
    if pd.isna(V): 
        return None
    TXT = str(V)
    for TOKEN in ["LR2020","LR2021","SR2020","SR2021"]:
        if TOKEN in TXT:
            return TOKEN
    return None

# detectar columnas clave
def DETECTAR_COLUMNAS(TRAIN: pd.DataFrame) -> Dict[str, object]:
    COLS_LOWER = [c.lower() for c in TRAIN.columns]
    ID_CANDIDATES = [c for c in TRAIN.columns if c.lower() in ["id","image_id","imageid","img_id","filename","image"]]
    SEASON_CANDS = [c for c in TRAIN.columns if "season" in c.lower() or c in ["SR2020","LR2020","SR2021","LR2021"]]
    STAGE_CANDS = [c for c in TRAIN.columns if c.lower() in ["growth_stage","growthstage","stage","gs","growth"]]

    LABEL_COLS = [c for c in TRAIN.columns if c.upper() in DAMAGE_CODES]

    if not LABEL_COLS:
        def LOOKS_PERCENT(S: pd.Series) -> bool:
            if not pd.api.types.is_numeric_dtype(S):
                return False
            VALS = S.dropna().unique()
            if len(VALS) == 0: 
                return False
            ALLOWED = set(range(0,101,10))
            SAMPLE = set(np.random.choice(VALS, size=min(200, len(VALS)), replace=False))
            RATIO = sum(v in ALLOWED for v in SAMPLE)/len(SAMPLE)
            return RATIO > 0.7
        CAND_PCT = [c for c in TRAIN.columns if LOOKS_PERCENT(TRAIN[c])]
        LIKELY_DAMAGE = [c for c in CAND_PCT if c.upper() == c and 1 < len(c) <= 3]
        LABEL_COLS = LIKELY_DAMAGE if LIKELY_DAMAGE else CAND_PCT

    SEASON_INFERRED_COL = None
    if not SEASON_CANDS:
        ID_COL = ID_CANDIDATES[0] if ID_CANDIDATES else TRAIN.columns[0]
        TEMP = TRAIN[ID_COL].map(EXTRAER_SEASON_DESDE_ID)
        if TEMP.notna().any():
            SEASON_INFERRED_COL = "SEASON_INFERRED"
    if not STAGE_CANDS:
        POSS = []
        for C in TRAIN.columns:
            VALS = set(map(str, TRAIN[C].dropna().unique()[:50]))
            if VALS and (VALS <= GROWTH_SET_SHORT or VALS <= set(GROWTH_MAP.values())):
                POSS.append(C)
        STAGE_CANDS = POSS

    # devolver las detecciones
    return {
        "ID_CANDIDATES": ID_CANDIDATES,
        "SEASON_CANDIDATES": SEASON_CANDS,
        "SEASON_INFERRED_COL": SEASON_INFERRED_COL,
        "STAGE_CANDIDATES": STAGE_CANDS,
        "LABEL_COLS": LABEL_COLS
    }

# resumen de columnas
TRAIN_OV = COLS_OVERVIEW(TRAIN_DF)
display(TRAIN_OV)

# detectar columnas clave
DETECCIONES = DETECTAR_COLUMNAS(TRAIN_DF)
print(json.dumps(DETECCIONES, indent=2))


,COLUMN,DTYPE,NON_NULL,NULLS,UNIQUE,EXAMPLE
0,extent,int64,26068,0,11,0
1,ID,object,26068,0,26068,ID_1S8OOWQYCB
2,damage,object,26068,0,8,WD
3,filename,object,26068,0,26068,L427F01330C01S03961Rp02052.jpg
4,growth_stage,object,26068,0,4,S
5,season,object,26068,0,4,SR2020


{
  "ID_CANDIDATES": [
    "ID",
    "filename"
  ],
  "SEASON_CANDIDATES": [
    "season"
  ],
  "SEASON_INFERRED_COL": null,
  "STAGE_CANDIDATES": [
    "growth_stage"
  ],
  "LABEL_COLS": [
    "extent"
  ]
}


# DICCIONARIO DE DATOS (TRAIN Y TEST) + % FALTANTES + DISCREPANCIAS

In [66]:
# limpieza y estandarización
def _DATA_PROFILE(DF: pd.DataFrame, DATASET_NAME: str) -> pd.DataFrame:
    """Perfil por columna: tipos, faltantes, %faltantes, únicos, ejemplo."""
    rows = len(DF)
    out = []
    for col in DF.columns:
        s = DF[col]
        non_null = int(s.notna().sum())
        nulls = int(s.isna().sum())
        unique = int(s.nunique(dropna=True))
        pct_missing = nulls / max(1, (nulls + non_null))
        example = s.dropna().iloc[0] if non_null > 0 else None
        out.append({
            "DATASET": DATASET_NAME,
            "COLUMN": col,
            "DTYPE": str(s.dtype),
            "ROWS": rows,
            "NON_NULL": non_null,
            "NULLS": nulls,
            "PCT_MISSING": round(pct_missing, 6),
            "UNIQUE": unique,
            "EXAMPLE": example
        })
    return pd.DataFrame(out)

# p erfiles individuales
DD_TRAIN = _DATA_PROFILE(TRAIN_DF, "TRAIN")
DD_TEST  = _DATA_PROFILE(TEST_DF,  "TEST")

print("=== DICCIONARIO – TRAIN ===")
display(DD_TRAIN.sort_values(["COLUMN"]))
print("=== DICCIONARIO – TEST ===")
display(DD_TEST.sort_values(["COLUMN"]))

# unión y discrepancias
def _as_str(x):
    return "" if x is None else str(x)

N_ROWS_TRAIN = len(TRAIN_DF)
N_ROWS_TEST  = len(TEST_DF)

all_cols = sorted(set(DD_TRAIN["COLUMN"]).union(set(DD_TEST["COLUMN"])))
rows = []
for col in all_cols:
    tr = DD_TRAIN[DD_TRAIN["COLUMN"] == col]
    te = DD_TEST[DD_TEST["COLUMN"] == col]

    dtype_train = tr["DTYPE"].iloc[0] if len(tr) else None
    dtype_test  = te["DTYPE"].iloc[0] if len(te) else None

    unique_train = int(tr["UNIQUE"].iloc[0]) if len(tr) else None
    unique_test  = int(te["UNIQUE"].iloc[0]) if len(te) else None

    pct_miss_train = float(tr["PCT_MISSING"].iloc[0]) if len(tr) else None
    pct_miss_test  = float(te["PCT_MISSING"].iloc[0]) if len(te) else None

    # Flags de presencia
    presente_train = bool(len(tr))
    presente_test  = bool(len(te))

    # Tipos iguales solo si ambos presentes y coinciden
    tipos_iguales = (presente_train and presente_test and (dtype_train == dtype_test))

    # ¿Es clave única en cada dataset?
    is_key_train = (unique_train == N_ROWS_TRAIN) if unique_train is not None else None
    is_key_test  = (unique_test  == N_ROWS_TEST)  if unique_test  is not None else None

    # Tasas de unicidad 
    unique_rate_train = (unique_train / N_ROWS_TRAIN) if unique_train is not None else None
    unique_rate_test  = (unique_test  / N_ROWS_TEST)  if unique_test  is not None else None

    # si es clave se considera que no hay problema
    # si falta en uno de los datasets, no aplica
    if (unique_train is None) or (unique_test is None):
        unique_dif_useful = None
    else:
        if is_key_train or is_key_test:
            unique_dif_useful = False
        else:
            unique_dif_useful = (unique_train != unique_test)

    rows.append({
        "COLUMN": col,
        "PRESENTE_TRAIN": presente_train,
        "PRESENTE_TEST":  presente_test,
        "DTYPE_TRAIN": _as_str(dtype_train),
        "DTYPE_TEST":  _as_str(dtype_test),
        "TIPOS_IGUALES": tipos_iguales,
        "UNIQUE_TRAIN": unique_train,
        "UNIQUE_TEST":  unique_test,
        "UNIQUE_RATE_TRAIN": unique_rate_train,
        "UNIQUE_RATE_TEST":  unique_rate_test,
        "IS_KEY_TRAIN": is_key_train,
        "IS_KEY_TEST":  is_key_test,
        "UNIQUE_DIF_USEFUL": unique_dif_useful,   
        "PCT_MISSING_TRAIN": pct_miss_train,
        "PCT_MISSING_TEST":  pct_miss_test,
    })

DD_MERGED = pd.DataFrame(rows).sort_values(["COLUMN"]).reset_index(drop=True)

print("=== DICCIONARIO – RESUMEN CON DISCREPANCIAS ===")
display(DD_MERGED)

# guardar en csv
OUT_PATH = OUTPUT_DIR / "data_dictionary.csv"
DD_MERGED.to_csv(OUT_PATH, index=False)
print("DICCIONARIO GUARDADO EN:", OUT_PATH.resolve())


=== DICCIONARIO – TRAIN ===


,DATASET,COLUMN,DTYPE,ROWS,NON_NULL,NULLS,PCT_MISSING,UNIQUE,EXAMPLE
0,TRAIN,ID,object,26068,26068,0,0.0,26068,ID_1S8OOWQYCB
3,TRAIN,damage,object,26068,26068,0,0.0,8,WD
4,TRAIN,extent,int64,26068,26068,0,0.0,11,0
1,TRAIN,filename,object,26068,26068,0,0.0,26068,L427F01330C01S03961Rp02052.jpg
2,TRAIN,growth_stage,object,26068,26068,0,0.0,4,S
5,TRAIN,season,object,26068,26068,0,0.0,4,SR2020


=== DICCIONARIO – TEST ===


,DATASET,COLUMN,DTYPE,ROWS,NON_NULL,NULLS,PCT_MISSING,UNIQUE,EXAMPLE
0,TEST,ID,object,8663,8663,0,0.0,8663,ID_ROOWKB90UZ
3,TEST,damage,object,8663,8663,0,0.0,8,WD
1,TEST,filename,object,8663,8663,0,0.0,8663,L122F00315C01S02151Rp04021.jpg
2,TEST,growth_stage,object,8663,8663,0,0.0,4,V
4,TEST,season,object,8663,8663,0,0.0,4,SR2020


=== DICCIONARIO – RESUMEN CON DISCREPANCIAS ===


,COLUMN,PRESENTE_TRAIN,PRESENTE_TEST,DTYPE_TRAIN,DTYPE_TEST,TIPOS_IGUALES,UNIQUE_TRAIN,UNIQUE_TEST,UNIQUE_RATE_TRAIN,UNIQUE_RATE_TEST,IS_KEY_TRAIN,IS_KEY_TEST,UNIQUE_DIF_USEFUL,PCT_MISSING_TRAIN,PCT_MISSING_TEST
0,ID,True,True,object,object,True,26068,8663.0,1.000000,1.000000,True,True,False,0.0,0.0
1,damage,True,True,object,object,True,8,8.0,0.000307,0.000923,False,False,False,0.0,0.0
2,extent,True,False,int64,,False,11,NaN,0.000422,NaN,False,None,None,0.0,NaN
3,filename,True,True,object,object,True,26068,8663.0,1.000000,1.000000,True,True,False,0.0,0.0
4,growth_stage,True,True,object,object,True,4,4.0,0.000153,0.000462,False,False,False,0.0,0.0
5,season,True,True,object,object,True,4,4.0,0.000153,0.000462,False,False,False,0.0,0.0


DICCIONARIO GUARDADO EN: C:\Users\andre\OneDrive\Desktop\PY2-DS\outputs\data_dictionary.csv


# PREPROCESAMIENTO DE DATOS

In [67]:
# función de preprocesamiento
def APLICAR_PREPROCESAMIENTO(DF: pd.DataFrame, IS_TRAIN: bool, DET: Dict[str, object]) -> pd.DataFrame:
    DF = DF.copy()

    # estandarizar nombres de columnas a mayúsculas
    DF.columns = [c.upper() for c in DF.columns]

    # identificar candidatas según detecciones
    ID_CANDS = [c.upper() for c in DET.get("ID_CANDIDATES", [])]
    SEASON_CANDS = [c.upper() for c in DET.get("SEASON_CANDIDATES", [])]
    STAGE_CANDS = [c.upper() for c in DET.get("STAGE_CANDIDATES", [])]
    LABEL_COLS = [c.upper() for c in DET.get("LABEL_COLS", []) if c.upper() in DF.columns]

    # 3) renombrar o extraer temporada
    if not SEASON_CANDS:
        ID_COL = ID_CANDS[0] if ID_CANDS else DF.columns[0]
        DF["SEASON"] = DF[ID_COL].map(EXTRAER_SEASON_DESDE_ID)
    else:
        SEASON_COL = SEASON_CANDS[0]
        DF.rename(columns={SEASON_COL: "SEASON"}, inplace=True)

    # normalizar valores de la etapa
    if STAGE_CANDS:
        STAGE_COL = STAGE_CANDS[0]
        DF.rename(columns={STAGE_COL: "GROWTH_STAGE_RAW"}, inplace=True)
    else:
        DF["GROWTH_STAGE_RAW"] = np.nan

    # mapear a código corto + nombre
    def MAP_TO_SHORT(v):
        if pd.isna(v): return np.nan
        s = str(v).strip().upper()
        if s in {"F","M","S","V"}: 
            return s
        return {"FLOWERING":"F","MATURITY":"M","SOWING":"S","VEGETATIVE":"V"}.get(s, np.nan)

    DF["GROWTH_STAGE"] = DF["GROWTH_STAGE_RAW"].map(MAP_TO_SHORT)
    DF["GROWTH_STAGE_NAME"] = DF["GROWTH_STAGE"].map(lambda x: GROWTH_MAP.get(x, np.nan))

    # asegurar columnas de etiquetas numéricas entre 0 y 100
    for C in LABEL_COLS:
        DF[C] = pd.to_numeric(DF[C], errors="coerce")
        DF[C] = DF[C].clip(lower=0, upper=100)

    # calcular etiqueta dominante solo en TRAIN y si existen las columnas ID, DAMAGE y EXTENT
    if IS_TRAIN and all(c in DF.columns for c in ["ID","DAMAGE","EXTENT"]):
        def ELEGIR_DOMINANTE(G):
            MX = G["EXTENT"].max()
            CAND = G[G["EXTENT"] == MX]

            NON_G = CAND[CAND["DAMAGE"] != "G"]
            CHOICE = NON_G.iloc[0] if len(NON_G) > 0 else CAND.iloc[0]
            return pd.Series({"DOMINANT_DAMAGE": CHOICE["DAMAGE"]})

        DOM = DF.groupby("ID", as_index=True).apply(ELEGIR_DOMINANTE)
        DF = DF.join(DOM, on="ID") 

    DF.attrs["LABEL_COLS"] = LABEL_COLS
    return DF

TRAIN_CLEAN = APLICAR_PREPROCESAMIENTO(TRAIN_DF, IS_TRAIN=True, DET=DETECCIONES)
TEST_CLEAN  = APLICAR_PREPROCESAMIENTO(TEST_DF,  IS_TRAIN=False, DET=DETECCIONES)

print("COLUMNAS (TRAIN_CLEAN):", list(TRAIN_CLEAN.columns)[:20], "...")
print("LABEL_COLS DETECTADAS:", TRAIN_CLEAN.attrs.get("LABEL_COLS", []))

display(TRAIN_CLEAN.head(3))
display(TEST_CLEAN.head(3))

# versión limpia
TRAIN_CLEAN.to_csv(OUTPUT_DIR / "TRAIN_CLEAN.csv", index=False)
TEST_CLEAN.to_csv(OUTPUT_DIR / "TEST_CLEAN.csv", index=False)
print("GUARDADO TRAIN_CLEAN/TEST_CLEAN EN:", OUTPUT_DIR.resolve())


COLUMNAS (TRAIN_CLEAN): ['ID', 'FILENAME', 'GROWTH_STAGE_RAW', 'DAMAGE', 'EXTENT', 'SEASON', 'GROWTH_STAGE', 'GROWTH_STAGE_NAME', 'DOMINANT_DAMAGE'] ...
LABEL_COLS DETECTADAS: ['EXTENT']


,ID,FILENAME,GROWTH_STAGE_RAW,DAMAGE,EXTENT,SEASON,GROWTH_STAGE,GROWTH_STAGE_NAME,DOMINANT_DAMAGE
0,ID_1S8OOWQYCB,L427F01330C01S03961Rp02052.jpg,S,WD,0,SR2020,S,Sowing,WD
1,ID_0MD959MIZ0,L1083F00930C39S12674Ip.jpg,V,G,0,SR2021,V,Vegetative,G
2,ID_JRJCI4Q11V,24_initial_1_1463_1463.JPG,V,G,0,LR2020,V,Vegetative,G


,ID,FILENAME,GROWTH_STAGE_RAW,DAMAGE,SEASON,GROWTH_STAGE,GROWTH_STAGE_NAME
0,ID_ROOWKB90UZ,L122F00315C01S02151Rp04021.jpg,V,WD,SR2020,V,Vegetative
1,ID_PTEDRY0CYM,L1089F03254C01S08845Rp25119.jpg,F,WD,LR2021,F,Flowering
2,ID_5WJXDV96R4,L365F01913C39S12578Rp42918.jpg,V,WD,SR2021,V,Vegetative


GUARDADO TRAIN_CLEAN/TEST_CLEAN EN: C:\Users\andre\OneDrive\Desktop\PY2-DS\outputs


# EDA: RESÚMENES, FALTANTES, CATEGÓRICAS, CRUCES, ATÍPICOS, CORRELACIONES

In [68]:
# análisis exploratorio
def RESUMEN_BÁSICO(DF: pd.DataFrame, NOMBRE: str):
    print(f"=== RESUMEN: {NOMBRE} ===")
    print("SHAPE:", DF.shape)
    print("\nTIPOS:")
    display(DF.dtypes.to_frame("DTYPE"))
    print("\nVALORES FALTANTES:")
    MISS = DF.isna().sum()
    display(MISS[MISS > 0].sort_values(ascending=False).to_frame("MISSING"))
    print("\nVISTA RÁPIDA:")
    display(DF.head(5))

# resumen básico
RESUMEN_BÁSICO(TRAIN_CLEAN, "TRAIN_CLEAN")
RESUMEN_BÁSICO(TEST_CLEAN,  "TEST_CLEAN")

# variables y observaciones 
print("NÚMERO DE VARIABLES (TRAIN):", TRAIN_CLEAN.shape[1])
print("NÚMERO DE OBSERVACIONES (TRAIN):", TRAIN_CLEAN.shape[0])

# separar numéricas y categóricas
NUM_COLS = [c for c in TRAIN_CLEAN.columns if pd.api.types.is_numeric_dtype(TRAIN_CLEAN[c])]
CAT_COLS = [c for c in TRAIN_CLEAN.columns if c not in NUM_COLS]

print("\nNUM_COLS (EJEMPLO):", NUM_COLS[:15], "...")
print("CAT_COLS (EJEMPLO):", CAT_COLS[:15], "...")

# estadísticas descriptivas para numéricas
if NUM_COLS:
    DESC_NUM = TRAIN_CLEAN[NUM_COLS].describe(percentiles=[0.25, 0.5, 0.75]).T
    display(DESC_NUM)
else:
    print("[AVISO] NO HAY COLUMNAS NUMÉRICAS PARA DESCRIBIR.")

# tabla de frecuencias para categóricas clave
def FREQ_TAB(DF, COL):
    if COL not in DF.columns:
        print(f"[AVISO] NO EXISTE {COL}")
        return
    VC = DF[COL].value_counts(dropna=False)
    display(VC.to_frame("COUNT"))

for COL in ["SEASON", "GROWTH_STAGE", "GROWTH_STAGE_NAME", "DOMINANT_DAMAGE"]:
    FREQ_TAB(TRAIN_CLEAN, COL)

# cruce de dos variables clave
def CROSS_TAB_COUNT(DF, A, B):
    if A in DF.columns and B in DF.columns:
        CT = pd.crosstab(DF[A], DF[B], dropna=False)
        display(CT)
        return CT
    else:
        print(f"[AVISO] NO SE PUEDE CRUZAR {A} × {B} (FALTAN COLUMNAS).")

CT1 = CROSS_TAB_COUNT(TRAIN_CLEAN, "GROWTH_STAGE", "SEASON")
CT2 = CROSS_TAB_COUNT(TRAIN_CLEAN, "DOMINANT_DAMAGE", "SEASON")

# gráficos exploratorios 
LABEL_COLS = TRAIN_CLEAN.attrs.get("LABEL_COLS", [])

# histogramas 
for C in LABEL_COLS[:8]:
    plt.figure()
    TRAIN_CLEAN[C].dropna().hist(bins=11)  
    plt.title(f"HISTOGRAMA – {C} (TRAIN)")
    plt.xlabel("EXTENSIÓN (%)")
    plt.ylabel("FRECUENCIA")
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / f"HIST_{C}.png")
    plt.close()

# diagrama de caja
for C in LABEL_COLS[:8]:
    plt.figure()
    plt.boxplot([TRAIN_CLEAN[C].dropna()], labels=[C], vert=True)
    plt.title(f"BOXPLOT – {C} (TRAIN)")
    plt.ylabel("EXTENSIÓN (%)")
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / f"BOX_{C}.png")
    plt.close()

# barras de frecuencia para categóricas clave
def PLOT_BAR_COUNTS(DF, COL, TITLE, FNAME):
    if COL not in DF.columns:
        print(f"[AVISO] NO EXISTE {COL} PARA GRAFICAR.")
        return
    VC = DF[COL].value_counts(dropna=False)
    plt.figure()
    plt.bar(VC.index.astype(str), VC.values)
    plt.title(TITLE)
    plt.xlabel(COL)
    plt.ylabel("COUNT")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / FNAME)
    plt.close()

PLOT_BAR_COUNTS(TRAIN_CLEAN, "SEASON", "DISTRIBUCIÓN POR TEMPORADA (TRAIN)", "BAR_SEASON.png")
PLOT_BAR_COUNTS(TRAIN_CLEAN, "GROWTH_STAGE", "DISTRIBUCIÓN POR ETAPA (TRAIN)", "BAR_STAGE.png")
PLOT_BAR_COUNTS(TRAIN_CLEAN, "DOMINANT_DAMAGE", "DISTRIBUCIÓN DAÑO DOMINANTE (TRAIN)", "BAR_DOMINANT.png")

# remplazo de IQR por percentil P95 por daño
def OUTLIERS_PERCENTILE_BY_DAMAGE(DF: pd.DataFrame, COL: str = "EXTENT", DAMAGE_COL: str = "DAMAGE",
                                  Q: float = 0.95, NONZERO_ONLY: bool = True) -> pd.Series:
    FLAGS = pd.Series(False, index=DF.index)
    if COL not in DF.columns or DAMAGE_COL not in DF.columns:
        return FLAGS

    for DMG, G in DF.groupby(DAMAGE_COL):
        X = G[COL]
        if NONZERO_ONLY:
            X = X[X > 0] 
        if X.empty:
            continue
        THR = X.quantile(Q)
        FLAGS.loc[G.index] = G[COL] > THR
    return FLAGS

# calcular outliers P95 por daño solo si existen EXTENT y DAMAGE
if "EXTENT" in TRAIN_CLEAN.columns and "DAMAGE" in TRAIN_CLEAN.columns:
    TRAIN_CLEAN["OUTLIER_P95"] = OUTLIERS_PERCENTILE_BY_DAMAGE(
        TRAIN_CLEAN, COL="EXTENT", DAMAGE_COL="DAMAGE", Q=0.95, NONZERO_ONLY=True
    )
    OUT_SUMMARY = (
        TRAIN_CLEAN
        .groupby("DAMAGE")["OUTLIER_P95"]
        .agg(SUM_OUTLIERS="sum", COUNT="count")
        .assign(RATE=lambda d: d["SUM_OUTLIERS"] / d["COUNT"])
        .sort_values("RATE", ascending=False)
    )
    display(OUT_SUMMARY)
    print("TOTAL OUTLIERS (P95, NONZERO ONLY):", int(TRAIN_CLEAN["OUTLIER_P95"].sum()))
else:
    print("[AVISO] NO SE PUDO CALCULAR OUTLIERS P95 (FALTAN COLUMNAS EXTENT/DAMAGE).")

# correlaciones entre etiquetas
if LABEL_COLS:
    CORR = TRAIN_CLEAN[LABEL_COLS].corr()
    display(CORR)

    plt.figure()
    plt.imshow(CORR.values, aspect="auto")
    plt.colorbar()
    plt.xticks(ticks=range(len(LABEL_COLS)), labels=LABEL_COLS, rotation=45, ha="right")
    plt.yticks(ticks=range(len(LABEL_COLS)), labels=LABEL_COLS)
    plt.title("MATRIZ DE CORRELACIÓN (LABELS)")
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / "CORR_LABELS.png")
    plt.close()
else:
    print("[AVISO] NO HAY LABEL_COLS PARA CALCULAR CORRELACIÓN.")

print("GRÁFICOS GUARDADOS EN:", OUTPUT_DIR.resolve())


=== RESUMEN: TRAIN_CLEAN ===
SHAPE: (26068, 9)

TIPOS:


,DTYPE
ID,object
FILENAME,object
GROWTH_STAGE_RAW,object
DAMAGE,object
EXTENT,int64
SEASON,object
GROWTH_STAGE,object
GROWTH_STAGE_NAME,object
DOMINANT_DAMAGE,object



VALORES FALTANTES:


,MISSING



VISTA RÁPIDA:


...

SEASON,LR2020,LR2021,SR2020,SR2021
DOMINANT_DAMAGE,,,,
DR,391,1413,1146,1566
DS,72,22,10,11
FD,0,3,0,10
G,845,3468,2447,4863
ND,101,38,111,22
PS,36,54,45,119
WD,588,2938,2401,3311
WN,0,9,3,25


,SUM_OUTLIERS,COUNT,RATE
DAMAGE,,,
DR,45,4516,0.009965
DS,0,115,0.000000
FD,0,13,0.000000
G,0,11623,0.000000
ND,0,272,0.000000
PS,0,254,0.000000
WD,0,9238,0.000000
WN,0,37,0.000000


TOTAL OUTLIERS (P95, NONZERO ONLY): 45


,EXTENT
EXTENT,1.0


GRÁFICOS GUARDADOS EN: C:\Users\andre\OneDrive\Desktop\PY2-DS\outputs


# RESÚMENES PROGRAMÁTICOS Y HALLAZGOS

In [69]:
# resumen de hallazgos
def RESUMEN_HALLAZGOS(TRAIN: pd.DataFrame):
    LBL = TRAIN.attrs.get("LABEL_COLS", [])
    LBL = [c for c in LBL if c in TRAIN.columns]

    LINEAS = []
    LINEAS.append("## 6. Conclusiones – Hallazgos principales\n")

    # faltantes
    MISS = TRAIN.isna().sum()
    MISS = MISS[MISS>0].sort_values(ascending=False)
    if not MISS.empty:
        TOP_MISS = MISS.head(8).to_dict()
        LINEAS.append("**Valores faltantes (top):** " + ", ".join([f"{k}: {v}" for k,v in TOP_MISS.items()]))

    # distribuciones
    if "SEASON" in TRAIN.columns:
        SEASON_VC = TRAIN["SEASON"].value_counts(dropna=False).to_dict()
        LINEAS.append("**Distribución por temporada:** " + ", ".join([f"{k}: {v}" for k,v in SEASON_VC.items()]))

    if "GROWTH_STAGE" in TRAIN.columns:
        STAGE_VC = TRAIN["GROWTH_STAGE"].value_counts(dropna=False).to_dict()
        LINEAS.append("**Distribución por etapa:** " + ", ".join([f"{k}: {v}" for k,v in STAGE_VC.items()]))

    # daño dominante
    if "DOMINANT_DAMAGE" in TRAIN.columns:
        DOM_VC = TRAIN["DOMINANT_DAMAGE"].value_counts(dropna=False).to_dict()
        LINEAS.append("**Daño dominante (conteos):** " + ", ".join([f"{k}: {v}" for k,v in DOM_VC.items()]))

    # estadísticas de etiquetas
    if LBL:
        MEANS = TRAIN[LBL].mean().sort_values(ascending=False).to_dict()
        LINEAS.append("**Promedios de extensión por etiqueta (mayor→menor):** " + ", ".join([f"{k}: {round(v,2)}%" for k,v in MEANS.items()]))

    TEXTO = "\n".join(LINEAS)
    print(TEXTO)

RESUMEN_HALLAZGOS(TRAIN_CLEAN)


## 6. Conclusiones – Hallazgos principales

**Distribución por temporada:** SR2021: 9927, LR2021: 7945, SR2020: 6163, LR2020: 2033
**Distribución por etapa:** V: 10015, M: 6664, F: 6164, S: 3225
**Daño dominante (conteos):** G: 11623, WD: 9238, DR: 4516, ND: 272, PS: 254, DS: 115, WN: 37, FD: 13
**Promedios de extensión por etiqueta (mayor→menor):** EXTENT: 7.1%
